In [80]:
import json
import pandas as pd
import numpy as np
import os

FIELD_LENGTH = 105.0  # unit: meters
FIELD_WIDTH = 68.0  # unit: meters
GOAL_WIDTH = 7.32  # unit: meters
PENALTY_X = 105.0/2-16.5 # left point (unit: meters)
PENALTY_Y = 40.32 # upper point (unit: meters)

# 文件路径
statsbomb_match_id = 3894907
statsbomb_event_path = "/home/z_chen/workspace3/laliga/laliga_23/statsbomb/events/3894907.csv"
skillcorner_tracking_path = "/home/z_chen/workspace3/laliga/laliga_23/skillcorner_v2/tracking/1553748.json"
skillcorner_match_path = "/home/z_chen/workspace3/laliga/laliga_23/skillcorner_v2/match/1553748.json"

# Check if the file exists
if not os.path.exists(statsbomb_event_path):
    raise FileNotFoundError(f"Statsbomb event file not found: {statsbomb_event_path}")
if not os.path.exists(skillcorner_tracking_path):
    raise FileNotFoundError(f"Skillcorner tracking file not found: {skillcorner_tracking_path}")
if not os.path.exists(skillcorner_match_path):
    raise FileNotFoundError(f"Skillcorner match file not found: {skillcorner_match_path}")

# 加载数据
events = pd.read_csv(statsbomb_event_path)

with open(skillcorner_tracking_path) as f:
    tracking = json.load(f)

with open(skillcorner_match_path, encoding='utf-8') as f:
    match = json.load(f)

# 球队名映射
team_name_dict = {
    'UD Almería': 'Almería', 'Real Sociedad': 'Real Sociedad', 'Athletic Club de Bilbao': 'Athletic Club', 
    'Villarreal CF': 'Villarreal', 'RC Celta de Vigo': 'Celta Vigo', 'Getafe CF': 'Getafe', 
    'UD Las Palmas': 'Las Palmas', 'Sevilla FC': 'Sevilla', 'Cadiz CF': 'Cádiz', 
    'Atlético Madrid': 'Atlético Madrid', 'RCD Mallorca': 'Mallorca', 'Valencia CF': 'Valencia', 
    'CA Osasuna': 'Osasuna', 'Girona FC': 'Girona', 'Real Betis Balompié': 'Real Betis', 
    'FC Barcelona': 'Barcelona', 'Deportivo Alavés': 'Deportivo Alavés', 'Granada CF': 'Granada', 
    'Rayo Vallecano': 'Rayo Vallecano', 'Real Madrid CF': 'Real Madrid'
}

home_team_name = team_name_dict[match['home_team']['name']]
away_team_name = team_name_dict[match['away_team']['name']]

team_dict = {
    match['home_team']['id']: {'role': 'home', 'name': home_team_name},
    match['away_team']['id']: {'role': 'away', 'name': away_team_name}
}

def rotate_around_center(x, y, center_x=105/2, center_y=34):

    # 将点平移，使旋转中心在原点
    translated_x = x - center_x
    translated_y = y - center_y

    # 围绕原点旋转180度，相当于取反
    rotated_x = -translated_x
    rotated_y = -translated_y

    # 将点移回原来的位置
    new_x = rotated_x + center_x
    new_y = rotated_y + center_y

    return new_x, new_y

# 将 trackable_objects 转换
trackable_objects = {}
home_count = away_count = 0
for player in match['players']:
    role = team_dict[player['team_id']]['role']
    position = player['player_role']['name']
    if role == 'home':
        trackable_objects[player['trackable_object']] = {
            'name': f"{player['first_name']} {player['last_name']}".strip(),
            'team': team_dict[player['team_id']]['name'],
            'role': role,
            'id': home_count,
            'position': position
        }
        home_count += 1
    elif role == 'away':
        trackable_objects[player['trackable_object']] = {
            'name': f"{player['first_name']} {player['last_name']}".strip(),
            'team': team_dict[player['team_id']]['name'],
            'role': role,
            'id': away_count,
            'position': position
        }
        away_count += 1

trackable_objects[match['ball']['trackable_object']] = {'name': 'ball', 'team': 'ball', 'role': 'ball', 'position': 'ball'}
ball_id = match['ball']['trackable_object']


# 创建事件数据 DataFrame
event_list = []
for _, event in events.iterrows():
    match_id = statsbomb_match_id
    event_period = event['period']
    event_time = event['timestamp']
    event_minute = event['minute']
    event_second = event['second']
    event_type = event['type']
    event_type_2 = None
    end_x = end_y = None
    pass_type = None
    pass_height = None 
    event_possession_team = event["possession_team"]
    pass_outcome = event.get("pass_outcome")
    pass_possibility = event.get("pass_pass_success_probability")
    pass_cluster_label = event.get("pass_pass_cluster_label")
    pass_cluster_possibility = event.get("pass_pass_cluster_probability")


    if event_type == "Pass":
        end_location = event.get('pass_end_location')
        if isinstance(end_location, str):
            end_location = [float(x) for x in end_location[1:-1].split(",")]
            end_x = round(end_location[0] * (1.05 / 1.2), 2)
            end_y = round(end_location[1] * (68 / 80), 2)
            # end_x, end_y = rotate_around_center(end_x, end_y)
        cross = event.get('pass_cross')
        pass_height = event.get('pass_height')
        pass_type = event.get('pass_type')
        
        # 使用字典替代 switch-case 逻辑
        event_type_2_mapping = {
            "Corner": "Corner",
            "Cross": "Cross",
            "High Pass": pass_height
        }
        
        if pass_type in event_type_2_mapping:
            event_type_2 = event_type_2_mapping[pass_type]
        elif cross and not pd.isna(cross):
            event_type_2 = "Cross"
        elif pass_height:
            event_type_2 = pass_height

    elif event_type == "Shot":
        event_type_2 = event.get('shot_outcome')

    event_team = event['team']
    home_team = 1 if event_team == home_team_name else 0
    event_player = event['player']
    event_location = event.get('location')

    if isinstance(event_location, str):
        event_location = [float(x) for x in event_location[1:-1].split(",")]
        start_x, start_y = round(event_location[0] * (1.05 / 1.2), 2), round(event_location[1] * (68 / 80), 2)
        # start_x, start_y = rotate_around_center(start_x, start_y)
    else:
        start_x = start_y = None

    time_components = event_time.split(':')
    event_seconds = round(float(time_components[0]) * 3600 + float(time_components[1]) * 60 + float(time_components[2]), 4) 
    if event_period == 2:
        event_seconds += 45 * 60
    elif event_period == 3:
        event_seconds += 90 * 60
    elif event_period == 4:
        event_seconds += (90 + 15) * 60

    # 添加事件数据
    event_list.append([match_id, event_period, event_time, event_minute, event_second, event_seconds, event_type, event_type_2, event_team, home_team, event_player, start_x, start_y, end_x, end_y, pass_type, pass_height, pass_outcome, pass_possibility, pass_cluster_label, pass_cluster_possibility,event_possession_team])

event_list_sorted = sorted(event_list, key=lambda x: x[5])

# 更新事件数据的列名
event_columns = [
    "match_id", "period", "time", "minute", "second", "seconds", "event_type", "event_type_2", "team", "home_team", 
    "player", "start_x", "start_y", "end_x", "end_y", "pass_type", "pass_height", "pass_outcome", 
    "pass_possibility", "pass_cluster_label", "pass_cluster_possibility", "possession_team"
]

# 创建事件数据 DataFrame
df_event = pd.DataFrame(event_list_sorted, columns=event_columns)

# 假设 df_event 已经存在并包含了事件数据
reverse_events = []

# 遍历每一个事件（从第一个到倒数第三个，因为我们需要后两个事件进行判断）
for i in range(len(df_event) - 2):
    pt1 = df_event.loc[i, 'possession_team']
    pt2 = df_event.loc[i + 1, 'possession_team']
    pt3 = df_event.loc[i + 2, 'possession_team']
    
    # 判断反转条件
    if pt1 != pt2:
        if pt2 == pt3:
            # 设置为需要反转
            df_event.loc[i, 'needs_reverse'] = True
        else:
            # 设置为反转且打印该事件的 seconds
            df_event.loc[i, 'needs_reverse'] = True
            reverse_events.append(df_event.loc[i, 'seconds'])
    else:
        # 设置为不需要反转
        df_event.loc[i, 'needs_reverse'] = False

# 打印需要反转的事件的 seconds
for event_seconds in reverse_events:
    # print(f"evetn2和event3再次反转的事件的时间: {seconds}")
    continue

# 对于最后两个事件，无法根据后两个事件判断，默认设置为不需要反转
df_event.loc[len(df_event) - 2:, 'needs_reverse'] = False



# 保存事件数据到 CSV 文件
event_output_csv_path = "/home/z_chen/workspace3/test/events.csv"
df_event.to_csv(event_output_csv_path, index=False)
print(f"事件 DataFrame 已保存为 CSV 文件：{event_output_csv_path}")


# 确定主队的方向并提取追踪数据
home_side = None
home_gk_x = away_gk_x = None
df_tracking_list = []

# 初始化跟踪数据列表
for frame in tracking:
    home_tracking = [None] * 2 * 23
    away_tracking = [None] * 2 * 23
    ball_x = ball_y = ball_z = None
    tracking_possession = frame.get("possession", {})
    possession_time = frame.get("timestamp", "")
    possession_period = frame.get("period", "")
    if possession_time:
        # 确保 possession_time 是字符串并尝试拆分为小时、分钟、秒
        try:
            time_parts = possession_time.split(':')
            possession_second = round(float(time_parts[0]) * 3600 + float(time_parts[1]) * 60 + float(time_parts[2]),4)
        except (ValueError, IndexError):
            # 捕捉转换失败或索引错误的异常，并设置默认值为 0
            possession_second = 0
    else:
        possession_second = 0
    possession_team = tracking_possession.get("group")
    tracking_data = frame["data"]

    if tracking_data:
        # 遍历每帧的对象数据
        for obj in tracking_data:
            track_obj = trackable_objects.get(obj['trackable_object'])
            if not track_obj:
                continue

            # 记录主客队球员位置
            if track_obj['role'] == 'home':
                home_tracking[2 * track_obj['id']] = round(obj['x'] + FIELD_LENGTH / 2,2)
                home_tracking[2 * track_obj['id'] + 1] = round(-obj['y'] + FIELD_WIDTH / 2 ,2)
            elif track_obj['role'] == 'away':
                away_tracking[2 * track_obj['id']] = round(obj['x'] + FIELD_LENGTH / 2,2)
                away_tracking[2 * track_obj['id'] + 1] = round(-obj['y'] + FIELD_WIDTH / 2 ,2)

            # 记录守门员位置
            if track_obj['position'] == "Goalkeeper":
                if track_obj['role'] == 'home' and home_gk_x is None:
                    home_gk_x = obj['x']
                elif track_obj['role'] == 'away' and away_gk_x is None:
                    away_gk_x = obj['x']

            # 记录球的位置
            if track_obj["role"] == "ball":
                ball_x = round(obj['x'] + FIELD_LENGTH / 2,2)
                ball_y = round(-obj['y'] + FIELD_WIDTH / 2 ,2)
                ball_z = round(obj["z"],2)
        # 一旦找到主客队守门员的坐标，进行判断并退出循环
        if home_gk_x is not None and away_gk_x is not None:
            home_side = 'left' if home_gk_x < away_gk_x else 'right'



    # # 应用 switch 逻辑到追踪数据
    # def switch_positions(home_tracking, away_tracking, ball_x, ball_y, switch_flag):
    #     if switch_flag:
    #         # 应用切换后的坐标转换
    #         for i in range(23):
    #             home_tracking[2 * i] = -home_tracking[2 * i] + FIELD_LENGTH / 2 if home_tracking[2 * i] is not None else None
    #             home_tracking[2 * i + 1] = home_tracking[2 * i + 1] + FIELD_WIDTH / 2 if home_tracking[2 * i + 1] is not None else None
    #             away_tracking[2 * i] = -away_tracking[2 * i] + FIELD_LENGTH / 2 if away_tracking[2 * i] is not None else None
    #             away_tracking[2 * i + 1] = away_tracking[2 * i + 1] + FIELD_WIDTH / 2 if away_tracking[2 * i + 1] is not None else None
    #         ball_x = -ball_x + FIELD_LENGTH / 2 if ball_x is not None else None
    #         ball_y = ball_y + FIELD_WIDTH / 2 if ball_y is not None else None
    #     else:
    #         # 应用不切换的坐标转换
    #         for i in range(23):
    #             home_tracking[2 * i] = home_tracking[2 * i] + FIELD_LENGTH / 2 if home_tracking[2 * i] is not None else None
    #             home_tracking[2 * i + 1] = -home_tracking[2 * i + 1] + FIELD_WIDTH / 2 if home_tracking[2 * i + 1] is not None else None
    #             away_tracking[2 * i] = away_tracking[2 * i] + FIELD_LENGTH / 2 if away_tracking[2 * i] is not None else None
    #             away_tracking[2 * i + 1] = -away_tracking[2 * i + 1] + FIELD_WIDTH / 2 if away_tracking[2 * i + 1] is not None else None
    #         ball_x = ball_x + FIELD_LENGTH / 2 if ball_x is not None else None
    #         ball_y = -ball_y + FIELD_WIDTH / 2 if ball_y is not None else None
    #     return home_tracking, away_tracking, ball_x, ball_y

    # # 确定是否需要切换并更新跟踪数据

    # switch_flag = (home_team == 1 and home_side == 'right' or home_team == 0 and home_side == 'left')

    # home_tracking, away_tracking, ball_x, ball_y = switch_positions(home_tracking, away_tracking, ball_x, ball_y, switch_flag)

    # 添加当前帧的跟踪数据
    df_tracking_list.append([possession_second, possession_period, possession_team, *home_tracking, *away_tracking, home_side, ball_x, ball_y, ball_z])


# 定义跟踪数据的 DataFrame 列名
home_tracking_columns = []
away_tracking_columns = []
for i in range(1, 24):
    home_tracking_columns.extend([f"h{i}_x", f"h{i}_y"])
    away_tracking_columns.extend([f"a{i}_x", f"a{i}_y"])
columns1 = ["seconds", "period","possession_team"] + home_tracking_columns + away_tracking_columns + ["home_side", "ball_x", "ball_y", "ball_z"]

df_tracking_list_sorted = sorted(df_tracking_list, key=lambda x: x[0])

# 创建跟踪数据 DataFrame
df_tracking = pd.DataFrame(df_tracking_list_sorted, columns=columns1)


# 保存追踪数据到 CSV 文件
tracking_output_csv_path = "/home/z_chen/workspace3/test/tracking.csv"
df_tracking.to_csv(tracking_output_csv_path, index=False)
print(f"追踪数据 DataFrame 已保存为 CSV 文件：{tracking_output_csv_path}")


事件 DataFrame 已保存为 CSV 文件：/home/z_chen/workspace3/test/events.csv
追踪数据 DataFrame 已保存为 CSV 文件：/home/z_chen/workspace3/test/tracking.csv


通过画图match_tracking_data,发现图已经没有问题了

In [ ]:
def ignore():   
    ball_frames = []
    for _, ball_frame in df_tracking.iterrows():
        b_seconds = ball_frame["seconds"]
        b_x = ball_frame["ball_x"]
        b_y = ball_frame["ball_y"]
        b_z = ball_frame["ball_z"]
        ball_frames.append([b_seconds,b_x,b_y,b_z])
        if b_seconds > 5:
            break


    ball_frames_secondhalf = []
    # 筛选 period 为 2 的所有行
    filtered_rows = df_tracking[df_tracking["period"] == 2]
    for _, ball_frame in filtered_rows.iterrows():
            b_seconds = ball_frame["seconds"]
            b_x = ball_frame["ball_x"]
            b_y = ball_frame["ball_y"]
            b_z = ball_frame["ball_z"]
            ball_frames_secondhalf.append([b_seconds,b_x,b_y,b_z])
            if b_seconds > 45*60+5:
                break


    h_columns_x = [col for col in df_tracking.columns if col.startswith("h") and col.endswith("_x")]
    h_columns_y = [col.replace("_x", "_y") for col in h_columns_x]

    h_players= []
    for _, h_row in df_tracking.iterrows():
        h_player = [(h_row[x], h_row[y]) for x, y in zip(h_columns_x, h_columns_y)]
        h_seconds = h_row["seconds"]
        h_players.append([h_player,h_seconds])

    a_columns_x = [col for col in df_tracking.columns if col.startswith("a") and col.endswith("_x")]
    a_columns_y = [col.replace("_x", "_y") for col in h_columns_x]

    a_players= []
    for index, a_row in df_tracking.iterrows():
        a_player = [(a_row[x], a_row[y]) for x, y in zip(h_columns_x, h_columns_y)]
        a_seconds = a_row["seconds"]
        a_players.append([a_player,a_seconds])


    def calculate_acceleration_and_max_seconds(data, player_position):
        
        # Extract timestamps, x, y, z coordinates
        m_seconds = [entry[0] for entry in data]
        m_x = np.array([entry[1] for entry in data])
        m_y = np.array([entry[2] for entry in data])
        m_z = np.array([entry[3] for entry in data])


        # Calculate differences
        delta_x = np.diff(m_x)
        delta_y = np.diff(m_y)
        delta_z = np.diff(m_z)
        delta_t = np.diff(m_seconds)

        # Calculate velocity components and magnitude
        vx = delta_x / delta_t
        vy = delta_y / delta_t
        vz = delta_z / delta_t
        velocity_magnitude = np.sqrt(vx**2 + vy**2 + vz**2)

        # Calculate acceleration
        delta_vx = np.diff(vx) / delta_t[1:]  # Acceleration in x direction
        delta_vy = np.diff(vy) / delta_t[1:]  # Acceleration in y direction
        delta_vz = np.diff(vz) / delta_t[1:]  # Acceleration in z direction
        acceleration_magnitude = np.sqrt(delta_vx**2 + delta_vy**2 + delta_vz**2)
        
        all_distances = []
        # Calculate distances between ball and player
        for index, second in enumerate(m_seconds):
            px = m_x[index]
            py = m_y[index]
            distances = []
            if player_position[1] == second:
                for _ , (kickoff_x, kickoff_y) in enumerate(player_position[0]): 
                    distance = np.sqrt((px - kickoff_x)**2 + (py - kickoff_y)**2)
                    distances.append(distance)
                distances = np.array([distances,second])  # Convert to numpy array

                valid_indices = np.where(distances <= 10)[0]
                if len(valid_indices) == 0:
                    return None, 0  # No valid timestamp found
                return  valid_indices
            
        # # Find the index of the maximum acceleration among valid indices



        max_acceleration_index_in_valid = np.argmax(acceleration_magnitude[valid_indices])
        max_acceleration_index = valid_indices[max_acceleration_index_in_valid]
        max_acceleration = acceleration_magnitude[max_acceleration_index]
        max_acceleration_seconds = m_seconds[max_acceleration_index + 2]  # +2 due to second order difference

        return max_acceleration_seconds, max_acceleration

    filtered = df_event[df_event["pass_type"] == "Kick Off"]

    print(filtered )
    if home_or_away == 1:
        max_seconds1,_ = calculate_acceleration_and_max_seconds(ball_frames,h_players)
        max_seconds2,_ = calculate_acceleration_and_max_seconds(ball_frames_secondhalf,h_players)
    else:
        max_seconds1,_ = calculate_acceleration_and_max_seconds(ball_frames,a_players)
        max_seconds2,_ = calculate_acceleration_and_max_seconds(ball_frames_secondhalf,a_players)


    # print(max_seconds1)
    # print(max_seconds2)

In [84]:
import numpy as np
import math

def calculate_acceleration_and_max_seconds(data, player_position):
    """
    Calculate acceleration and find the timestamp with the maximum acceleration.

    Parameters:
    - data: List of lists containing [timestamp, x, y, z] coordinates of the ball.
    - player_position: List of lists containing player positions and corresponding timestamps.

    Returns:
    - max_acceleration_seconds: The timestamp at which maximum acceleration occurs.
    - max_acceleration: The maximum acceleration value.
    """
    # Extract timestamps, x, y, z coordinates
    m_seconds = [entry[0] for entry in data]
    m_x = np.array([entry[1] for entry in data])
    m_y = np.array([entry[2] for entry in data])
    m_z = np.array([entry[3] for entry in data])

    # Calculate differences
    delta_x = np.diff(m_x)
    delta_y = np.diff(m_y)
    delta_z = np.diff(m_z)
    delta_t = np.diff(m_seconds)

    # Avoid division by zero
    delta_t[delta_t == 0] = 1e-6  # Use a small epsilon value to avoid NaN

    # Calculate velocity components and magnitude
    vx = delta_x / delta_t
    vy = delta_y / delta_t
    vz = delta_z / delta_t
    velocity_magnitude = np.sqrt(vx**2 + vy**2 + vz**2)

    # Calculate acceleration
    delta_vx = np.diff(vx) / delta_t[1:]  # Acceleration in x direction
    delta_vy = np.diff(vy) / delta_t[1:]  # Acceleration in y direction
    delta_vz = np.diff(vz) / delta_t[1:]  # Acceleration in z direction
    acceleration_magnitude = np.sqrt(delta_vx**2 + delta_vy**2 + delta_vz**2)

    max_acceleration = None
    max_acceleration_seconds = None

    # Calculate distances between ball and player
    for index, m_second in enumerate(m_seconds):
        px = m_x[index]
        py = m_y[index]
        distances = []
        for p_frame in player_position:
            p_seconds = p_frame[1]
            if p_seconds == m_second:
                for _, (kickoff_x, kickoff_y) in enumerate(p_frame[0]):
                    if kickoff_x is not None and kickoff_y is not None:
                        distance = np.sqrt((px - kickoff_x)**2 + (py - kickoff_y)**2)
                        distances.append(distance)
        
        distances = np.array(distances)  # Convert to numpy array
        valid_indices = np.where(distances <= 2)[0]
        if len(valid_indices) > 0 and len(acceleration_magnitude) > 0:
            max_acceleration_index_in_valid = np.argmax(acceleration_magnitude)
            max_acceleration = acceleration_magnitude[max_acceleration_index_in_valid]
            max_acceleration_seconds = m_seconds[max_acceleration_index_in_valid + 2]  # +2 due to second order difference
            break

    return max_acceleration_seconds if max_acceleration_seconds is not None else 0.0, max_acceleration if max_acceleration is not None else 0.0


filtered = df_event[df_event["pass_type"] == "Kick Off"]
filter1 = filtered[filtered["period"]== 1]
filter2 = filtered[filtered["period"]== 2]

home_or_away1 = filter1["home_team"].iloc[0]
home_or_away2 = filter2["home_team"].iloc[0]
print(home_or_away1)
print(home_or_away2)
ball_frames = []

# # first half 
# if home_or_away1 == 1:
#     for _, ball_frame in df_tracking.iterrows():
#         if np.isnan(ball_frame['ball_x']) or np.isnan(ball_frame['ball_y']):
#             continue
#         ball_team = ball_frame["possession_team"]
#         if ball_team == "home team":
#             b_seconds = ball_frame["seconds"]
#             b_x = ball_frame["ball_x"]
#             b_y = ball_frame["ball_y"]
#             b_z = ball_frame["ball_z"]
#             ball_frames.append([b_seconds, b_x, b_y, b_z])
#             if b_seconds > 5:
#                 break
# else:
#     for _, ball_frame in df_tracking.iterrows():
#         if np.isnan(ball_frame['ball_x']) or np.isnan(ball_frame['ball_y']):
#             continue
#         ball_team = ball_frame["possession_team"]
#         if ball_team == "away team":
#             b_seconds = ball_frame["seconds"]
#             b_x = ball_frame["ball_x"]
#             b_y = ball_frame["ball_y"]
#             b_z = ball_frame["ball_z"]
#             ball_frames.append([b_seconds, b_x, b_y, b_z])
#             if b_seconds > 5:
#                 break


# ball_frames_secondhalf = []
# # 筛选 period 为 2 的所有行

# if home_or_away2 == 1:
#     for _, second_frame in filtered_rows.iterrows():
#             sb_team = second_frame["possession_team"]
#             if sb_team == "home team":
#                 sb_seconds = second_frame["seconds"]
#                 sb_x = second_frame["ball_x"]
#                 sb_y = second_frame["ball_y"]
#                 sb_z = second_frame["ball_z"]
#                 ball_frames_secondhalf.append([sb_seconds,sb_x,sb_y,sb_z,possession_team])
#                 if sb_seconds > 45*60+10:
#                     break
# else:
#     for _, second_frame in filtered_rows.iterrows():
#             sb_team = second_frame["possession_team"]
#             if sb_team == "away team":
#                 sb_seconds = second_frame["seconds"]
#                 sb_x = second_frame["ball_x"]
#                 sb_y = second_frame["ball_y"]
#                 sb_z = second_frame["ball_z"]
#                 ball_frames_secondhalf.append([sb_seconds,sb_x,sb_y,sb_z,possession_team])
#                 if sb_seconds > 45*60+10:
#                     break  

for _, ball_frame in df_tracking.iterrows():
    if np.isnan(ball_frame['ball_x']) or np.isnan(ball_frame['ball_y']):
        continue
    b_seconds = ball_frame["seconds"]
    b_x = ball_frame["ball_x"]
    b_y = ball_frame["ball_y"]
    b_z = ball_frame["ball_z"]
    ball_frames.append([b_seconds, b_x, b_y, b_z])
    if b_seconds > 5:
        break


ball_frames_secondhalf = []
# 筛选 period 为 2 的所有行
filtered_rows = df_tracking[df_tracking["period"]==2]

for _, second_frame in filtered_rows.iterrows():
    if np.isnan(second_frame['ball_x']) or np.isnan(second_frame['ball_y']):
        continue
    sb_seconds = second_frame["seconds"]
    sb_x = second_frame["ball_x"]
    sb_y = second_frame["ball_y"]
    sb_z = second_frame["ball_z"]
    ball_frames_secondhalf.append([sb_seconds,sb_x,sb_y,sb_z,possession_team])
    # if sb_seconds > 45*60+5:
    if sb_seconds > 45*60+10:
        break


print(ball_frames)
print(ball_frames_secondhalf)

h_columns_x = [col for col in df_tracking.columns if col.startswith("h") and col.endswith("_x")]
h_columns_y = [col.replace("_x", "_y") for col in h_columns_x]

h_players = []
for _, h_row in df_tracking.iterrows():
    if all(pd.isna(h_row[x]) for x in h_columns_x):
        continue
    h_player = [(h_row[x], h_row[y]) for x, y in zip(h_columns_x, h_columns_y)]
    h_seconds = h_row["seconds"]
    h_players.append([h_player, h_seconds])

a_columns_x = [col for col in df_tracking.columns if col.startswith("a") and col.endswith("_x")]
a_columns_y = [col.replace("_x", "_y") for col in a_columns_x]

a_players = []
for _, a_row in df_tracking.iterrows():
    if all(pd.isna(a_row[x]) for x in a_columns_x):
        continue
    a_player = [(a_row[x], a_row[y]) for x, y in zip(a_columns_x, a_columns_y)]
    a_seconds = a_row["seconds"]
    a_players.append([a_player, a_seconds])

# # 检查 a_players 是否完全为空
# is_all_none_or_nan = all(
#     all(value is None or (isinstance(value, float) and math.isnan(value))
#         for value in row)
#     for row in a_players
# )

# if is_all_none_or_nan:
#     print("a_players is completely empty (all values are None or NaN).")
# else:
#     print("a_players is not completely empty.")

# Determine home or away and calculate max seconds
if home_or_away1==1:
    max_seconds1,max_acceleration1 = calculate_acceleration_and_max_seconds(ball_frames,h_players)

else:
    max_seconds1,max_acceleration1= calculate_acceleration_and_max_seconds(ball_frames,a_players)


if home_or_away2==1:
    max_seconds2,max_acceleration2= calculate_acceleration_and_max_seconds(ball_frames_secondhalf,h_players)
else:
    max_seconds2,max_acceleration2= calculate_acceleration_and_max_seconds(ball_frames_secondhalf,a_players)   

print(max_seconds1)
print(max_acceleration1)
print(max_seconds2)
print(max_acceleration2)




1
0
[[0.0, 53.63, 36.58, -0.22], [0.1, 54.13, 37.1, -0.28], [0.2, 55.94, 37.59, -0.36], [0.3, 56.46, 37.5, -0.35], [0.4, 56.76, 37.33, -0.33], [0.5, 56.97, 37.15, -0.31], [0.6, 57.17, 37.02, -0.3], [0.7, 57.32, 36.92, -0.29], [0.8, 57.47, 36.84, -0.29], [0.9, 57.62, 36.76, -0.29], [1.0, 57.77, 36.71, -0.28], [1.1, 57.92, 36.66, -0.28], [1.2, 58.07, 36.62, -0.27], [1.3, 58.22, 36.6, -0.26], [1.4, 58.39, 36.59, -0.25], [1.5, 58.57, 36.59, -0.24], [1.6, 58.77, 36.61, -0.23], [1.7, 59.0, 36.63, -0.22], [1.8, 59.25, 36.66, -0.2], [1.9, 59.52, 36.69, -0.19], [2.0, 59.82, 36.72, -0.17], [2.1, 60.15, 36.74, -0.14], [2.2, 60.53, 36.75, -0.08], [2.3, 60.93, 36.75, -0.03], [2.4, 61.47, 36.73, 0.06], [2.5, 61.97, 36.79, 0.06], [2.6, 62.88, 36.65, 0.11], [2.7, 63.5, 36.6, 0.14], [2.8, 64.04, 36.65, 0.17], [2.9, 65.09, 36.97, 0.16], [3.0, 66.11, 37.24, 0.17], [3.1, 66.97, 37.42, 0.19], [3.2, 67.75, 37.54, 0.21], [3.3, 68.49, 37.61, 0.22], [3.4, 69.15, 37.68, 0.22], [3.5, 69.75, 37.79, 0.23], [3.6, 7

In [87]:
# 保存原来的 seconds 列到 raw_seconds
df_tracking["raw_seconds"] = df_tracking["seconds"]

# 构造新的 adjusted_seconds 列
df_tracking["adjusted_seconds"] = df_tracking["seconds"]
df_tracking.loc[df_tracking["period"] == 1, "adjusted_seconds"] -= max_seconds1
df_tracking.loc[df_tracking["period"] == 2, "adjusted_seconds"] -= max_seconds2

# # 用 adjusted_seconds 覆盖 seconds
# df_tracking["seconds"] = df_tracking["adjusted_seconds"]
# # 删除 adjusted_seconds 列（可选）
# df_tracking.drop(columns=["adjusted_seconds"], inplace=True)

print(df_tracking["adjusted_seconds"].iloc[10])
print(df_tracking["raw_seconds"].iloc[10])


-0.3
0.0


In [ ]:
def distance_score(pass_event, player_period, ball_velocity_period):

    event_second = pass_event[0]    
    event_player_x = pass_event[1]
    event_player_y = pass_event[2]
    event_player_side = pass_event[3]

    ball_data_dict = {
        "second": ball_velocity_period[0], 
        "x": ball_velocity_period[1], 
        "y": ball_velocity_period[2]   
    }

    tracking_event_frame = [] # 存了该second下 pass和所有tracking球员的距离与时间
    ball_distances_frame = [] #该second下所有球员和ball的距离

    tracking_second = player_period[0]    
    tracking_player_x = player_period[1]        
    tracking_player_y = player_period[2]                                                                                    
    tracking_player_id = player_period[3]

    tracking_event_distance = ((event_player_x - tracking_player_x) ** 2 + (event_player_y - tracking_player_y) ** 2) ** 0.5
    tracking_event_frame.append((tracking_second, tracking_event_distance, tracking_player_id))

    ball_distance = ((ball_data_dict["x"] - event_player_x) ** 2 + (ball_data_dict["y"] - event_player_y) ** 2) ** 0.5
    ball_distances_frame.append((tracking_second, ball_distance, tracking_player_id))


    # 计算每个球员的综合得分
    scores = []
    weight_event = 0.5
    weight_ball = 0.5

    for i in range(len(tracking_event_frame)):
        # 获取对应的时间和ID
        tracking_time_event, event_distance, player_id_event = tracking_event_frame[i]
        tracking_time_ball, ball_distance, player_id_ball = ball_distances_frame[i]

        if tracking_time_event != tracking_time_ball or player_id_event != player_id_ball:
            continue

        # 计算综合得分
        combined_score = weight_event * (1 / (event_distance + 1e-6)) + weight_ball * (1 / (ball_distance + 1e-6))
        scores.append((tracking_time_event, player_id_event, combined_score))

        # 找出最高分
        if scores:
            best_score = max(scores, key=lambda x: x[2])
            return best_score

In [ ]:
def ignore():
    # Process tracking data
    # Kick Off 对齐。 注意df_list是dataframe
    ball_velocity_period_1 = []
    ball_velocity_period_2 = []
    tracking_dict = {}
    kick_off_events = df_event[df_event['pass_type'] == 'Kick Off']
    kick_off1 = kick_off_events[kick_off_events['period'] == 1]
    kick_off2 = kick_off_events[kick_off_events['period'] == 2]


    kick_off_player1 = []
    kick_off_player2 = []

    for _ , position1 in kick_off1.iterrows():
        kick_off_x1 = position1["start_x"]
        kick_off_y1 = position1["start_y"]
        kick_off_player1.append([kick_off_x1,kick_off_y1])


    for _ , position2 in kick_off2.iterrows():
        kick_off_x2 = position2["start_x"]
        kick_off_y2 = position2["start_y"]
        kick_off_player2.append([kick_off_x2,kick_off_y2])

    print(kick_off_player1)
    print(kick_off_player2)

    for _ , frame in df_tracking.iterrows():
        tracking_seconds = frame["seconds"]
        period = frame['period']
        if period==1 and seconds<=10:
            ball_x1 = frame["ball_x"]
            ball_y1 = frame["ball_y"]
            ball_z1 = frame["ball_z"]
            ball_velocity_period_1.append([tracking_seconds, ball_x1, ball_y1 ,ball_z1])

        if period==2 and seconds <= 45*60+10:
            ball_x2 = frame["ball_x"]
            ball_y2 = frame["ball_y"]
            ball_z2 = frame["ball_z"]
            ball_velocity_period_2.append([tracking_seconds, ball_x2, ball_y2 ,ball_z2])



    # 为什么ball的z轴有负数？
    if not ball_velocity_period_1 == [] or not ball_velocity_period_2 == []:
        print(ball_velocity_period_1)
        print(ball_velocity_period_2)
        try:
            max_velocity_seconds1, max_velocity1 = calculate_acceleration_and_max_timestamp(ball_velocity_period_1,kick_off_player1[0])
            # print(max_velocity_seconds1)
        except:
            max_velocity_seconds1 = -1
        
        try:
            max_velocity_seconds2, max_velocity2 = calculate_acceleration_and_max_timestamp(ball_velocity_period_2,kick_off_player2[0])
        except:
            max_velocity_seconds2 = -1
        
        if max_velocity_seconds1 == -1 and max_velocity_seconds2 != -1:
            max_velocity_seconds1 = max_velocity_seconds2
        elif max_velocity_seconds1 != -1 and max_velocity_seconds2 == -1:
            max_velocity_seconds2 = max_velocity_seconds1
        elif max_velocity_seconds1 == -1 and max_velocity_seconds2 == -1:
            max_velocity_seconds1 = max_velocity_seconds2 = 0

In [ ]:
# 找到 h_columns_x 和 h_columns_y 中的最大值和最小值
x_max = df_tracking[h_columns_x].max().max()  # 最大值
x_min = df_tracking[h_columns_x].min().min()  # 最小值

y_max = df_tracking[h_columns_y].max().max()  # 最大值
y_min = df_tracking[h_columns_y].min().min()  # 最小值

# 打印结果
print(f"X-columns max: {x_max}, min: {x_min}")
print(f"Y-columns max: {y_max}, min: {y_min}")


X-columns max: 109.76, min: -3.71
Y-columns max: 76.07, min: -2.68


In [89]:
# 使用 Pandas 过滤掉 NaN 值
filtered_df = df_event[['end_x', 'end_y']].dropna()

# 找出 x 和 y 的最小值和最大值
x_min = filtered_df['end_x'].min()
x_max = filtered_df['end_x'].max()
y_min = filtered_df['end_y'].min()
y_max = filtered_df['end_y'].max()

# 打印最小值和最大值
print(f"x 最小值: {x_min}, x 最大值: {x_max}")
print(f"y 最小值: {y_min}, y 最大值: {y_max}")


x 最小值: 1.66, x 最大值: 105.0
y 最小值: 0.09, y 最大值: 68.0


In [109]:
from tqdm import tqdm

# Filter pass events
pass_events = df_event[(df_event['event_type'] == 'Pass') | (df_event['event_type'] == 'Ball Receipt*') | (df_event['event_type'] == 'Carry')]
pass_events_seconds = pass_events["seconds"]

def get_window_of_frames_around(action_second, df_tracking, ta):
    """
    Gets a window of frames around the given action_second.
    """
    window = []
    for _, frame in df_tracking.iterrows():
        if frame["adjusted_seconds"] is not None and not pd.isna(frame["period"]):
            frame_seconds = frame['adjusted_seconds']
            if action_second - ta <= frame_seconds <= action_second + ta:
                window.append(frame)
    return window

# windows for all pass events
ta = 5
windows = []
for action_second in tqdm(pass_events_seconds[:20], desc="Processing windows"):
    if action_second is not None:
        window = get_window_of_frames_around(action_second, df_tracking[:1000], ta)
        if not window:
            print(f"No frames found around action_second={action_second}")
        windows.append(window)

print(windows)

Processing windows:   0%|          | 0/20 [00:00<?, ?it/s]

Processing windows: 100%|██████████| 20/20 [00:01<00:00, 15.94it/s]


[[seconds               0.0
period                1.0
possession_team      None
h1_x                  NaN
h1_y                  NaN
                    ...  
ball_x              53.63
ball_y              36.58
ball_z              -0.22
raw_seconds           0.0
adjusted_seconds     -0.3
Name: 10, Length: 101, dtype: object, seconds               0.1
period                1.0
possession_team      None
h1_x                  NaN
h1_y                  NaN
                    ...  
ball_x              54.13
ball_y               37.1
ball_z              -0.28
raw_seconds           0.1
adjusted_seconds     -0.2
Name: 20, Length: 101, dtype: object, seconds               0.2
period                1.0
possession_team      None
h1_x                  NaN
h1_y                  NaN
                    ...  
ball_x              55.94
ball_y              37.59
ball_z              -0.36
raw_seconds           0.2
adjusted_seconds     -0.1
Name: 21, Length: 101, dtype: object, seconds               0.3


In [ ]:
# 1）ball_velocity_periods
# 2）根据window[index]得到player_periods[key]
pass_player = pass_events["player"]
pass_player = pass_player.str.strip()
player_periods = {}
ball_velocity_periods = []
ball_id = match['ball']['trackable_object']


for player in tqdm(match["players"], desc="Processing players"):
    full_name = f"{player['first_name']} {player['last_name']}".strip()
    obj_id = player["trackable_object"]

    if full_name in pass_player:
        for index, window in enumerate(windows):
            # key = f"player_period_{index + 1}"
            key = index + 1
            if key not in player_periods:
                player_periods[key] = []

            for frame in window:
                # 获取时间戳和比赛阶段
                time = frame.get('timestamp')
                period = frame.get('period')
                data = frame.get('data', [])
                possession = frame.get("possession", {})  # 获取控球信息，默认空字典
                team_group = possession.get("group")  # 获取控球方

            # 仅处理 possession 不为空的帧
                if not team_group:
                    continue

                if time:
                    try:
                        time_components = time.split(':')
                        seconds = float(time_components[0]) * 3600 + float(time_components[1]) * 60 + float(time_components[2])
                    except (ValueError, IndexError):
                        continue

                    for obj in data:
                        if obj.get('trackable_object') == ball_id:
                            ball_velocity_periods.append([seconds, obj['x'], obj['y'], obj['z']])
                        elif obj.get('trackable_object') == obj_id:
                            player_periods[key].append([seconds, obj['x'], obj['y'], obj["trackable_object"],team_group, period])

    else:

        for index, window in enumerate(windows):
            # key = f"player_period_{index + 1}"
            key = index + 1
            if key not in player_periods:
                player_periods[key] = []  # 初始化 player_periods 的键

            for frame in window:
                # 获取控球信息
                possession = frame.get("possession", {})  # 控球信息
                team_group = possession.get("group")  # 控球方，可能为空

                # 仅处理 possession 不为空的帧
                if not team_group:
                    continue

                time = frame.get('timestamp')
                if time:
                    try:
                        time_components = time.split(':')
                        seconds = float(time_components[0]) * 3600 + float(time_components[1]) * 60 + float(time_components[2])
                    except (ValueError, IndexError):
                        print(f"Invalid timestamp format: {time}. Skipping frame.")
                        continue
                period = frame.get('period')
                data = frame.get('data', [])

                # 遍历当前帧中的跟踪对象数据
                for obj in data:
                    trackable_object = obj.get('trackable_object')

                    # 如果是球的 trackable_object
                    if trackable_object == ball_id:
                        ball_velocity_periods.append([seconds, obj['x'], obj['y'], obj.get('z', 0)])
                    # 如果是目标球员的 trackable_object
                    elif trackable_object == obj_id:
                        player_periods[key].append([seconds, obj['x'], obj['y'], trackable_object,team_group, period])



# 确保 pass_events 是一个 DataFrame
if not isinstance(pass_events, pd.DataFrame):
    raise ValueError("pass_events is not a Pandas DataFrame. Check data loading or filtering.")

# 检查是否有 'seconds' 列
if "seconds" not in pass_events.columns:
    raise KeyError("'seconds' column is missing in pass_events DataFrame.")

# pass_events_periods
pass_events_periods = []
for _, row in pass_events.iterrows():
    pass_second = row["seconds"]
    pass_x = row["start_x"]
    pass_y = row["start_y"]
    pass_event_team = row["home_team"]
    if pass_event_team == 1:
        pass_events_periods.append([pass_second,pass_x,pass_y,'home team'])
    else:
        pass_events_periods.append([pass_second,pass_x,pass_y,'away team'])



# synchronize the data by score
all_max_scores = []

for index, (pass_period, ball_velocity_period) in enumerate(zip(pass_events_periods, ball_velocity_periods)):
    pass_event_second = pass_period[0]
    pass_event_team = pass_period[3]
    ball_velocity_seconds = ball_velocity_period[0]
    ta = 5
    max_score = float('-inf')  # 初始化最大分数为负无穷
    max_score_info = []  # 用于存储最大分数对应的信息

    # 确保 player_periods 有效，并检查索引范围
    if index + 1 < len(player_periods):
        for i, player_period in enumerate(player_periods[index+1]):
            player_second = player_period[0]
            player_team = player_period[4]
            period_half = player_period[5]

            if player_second == ball_velocity_seconds:
                score = distance_score(pass_period, player_period, ball_velocity_period)[2]

                # 只保留当前最大分数的对应信息
                if score > max_score and pass_event_team == player_team:
                    max_score = score
                    max_score_info = [pass_event_second, player_second, max_score, period_half]
            else:
                score = [0,0,0,0]
        # 在遍历完所有的 player_period 之后，再添加最大分数信息
        if max_score_info:
            all_max_scores.append(max_score_info)
    else:
        all_max_scores.append([0, 0, 0, 0])
# for index, (pass_period, ball_velocity_period) in enumerate(zip(pass_events_periods, ball_velocity_periods)):
#     pass_event_second = pass_period[0]
#     pass_event_team = pass_period[3]
#     ta = 5
#     max_score = float('-inf')  # 初始化最大分数为负无穷
#     max_score_info = []  # 用于存储最大分数对应的信息

#     # 确保 player_periods 有效，并检查索引范围
#     if index + 1 < len(player_periods):
#         for i, player_period in enumerate(player_periods[index+1]):
#             player_second = player_period[0]
#             player_team = player_period[4]
#             period_half = player_period[5]

#             score = distance_score(pass_period, player_period, ball_velocity_period)[2]
#             if score > max_score and pass_event_team == player_team:
#                 max_score = score
#                 max_score_info = [pass_event_second, player_second, max_score, period_half]

#             if max_score_info:
#                 all_max_scores.append(max_score_info)

#     else:
#         all_max_scores.append([0,0,0,0])

# Prepare data for DataFrame
df_list1 = []
for index, event in events.iterrows():
    event_id = event['id']
    match_id = statsbomb_match_id
    period = event['period']
    time = event['timestamp']
    minute = event['minute']
    second = event['second']
    event_type = event['type']
    event_type_2 = None
    end_x = end_y = None
    if event_type == "Pass":
        end_location=event.get('pass_end_location')
        #check if end_location is a string
        if isinstance(end_location, (str)):
            end_location = [float(x) for x in end_location[1:-1].split(",")]
            end_x = end_location[0]
            end_y = end_location[1]
        cross=event.get('pass_cross')
        pass_height=event.get('pass_height')
        pass_type=event.get('pass_type')
        if pass_type=="Corner":
            event_type_2="Corner"
        elif cross and not np.isnan(cross):
            event_type_2="Cross"
        elif pass_height:
            event_type_2=pass_height
    elif event_type=="Shot":
        event_type_2=event.get('shot_outcome')

    team = event['team']
    home_team = 1 if team == home_team_name else 0
    player = event['player']
    location = event['location']

    if isinstance(location, str):
        location = [float(x) for x in location[1:-1].split(",")]
        start_x, start_y = location[0], location[1]
    else:
        start_x = start_y = None

    time_components = time.split(':')
    seconds = round(float(time_components[0]) * 3600 + float(time_components[1]) * 60 + float(time_components[2]), 4)
    if period == 2:
        seconds += 45 * 60
    elif period == 3:
        seconds += 90 * 60
    elif period == 4:
        seconds += (90 + 15) * 60


# 根据分数找到相应event最匹配的tracking的时间并写入trackable_objects
# 可能要改。 明天看一下max_score的结构
    for max_score_period in all_max_scores:
        
        if seconds == max_score_period[0]:
            secondsround = max_score_period[1]
            uid = f"{period}_{secondsround}"
            tracking_data = tracking_dict.get(uid)


        home_tracking = [None] * 2 * 23
        away_tracking = [None] * 2 * 23
        home_side = [None]
        ball_x = None
        ball_y = None

    if tracking_data:                
        for obj in tracking_data:
            track_obj = trackable_objects[obj['trackable_object']]
            if track_obj['role'] == 'home':
                home_tracking[2 * track_obj['id']] = obj['x']
                home_tracking[2 * track_obj['id'] + 1] = obj['y']
            elif track_obj['role'] == 'away':
                away_tracking[2 * track_obj['id']] = obj['x']
                away_tracking[2 * track_obj['id'] + 1] = obj['y']

            if track_obj['position'] == "Goalkeeper":
                if track_obj['role'] == 'home':
                    home_gk_x = obj['x']
                elif track_obj['role'] == 'away':
                    away_gk_x = obj['x']

            if track_obj["role"] == "ball":
                ball_x = obj["x"]
                ball_y = obj["y"]
        # Determine the side of the home team based on the goalkeeper's position
        if home_gk_x < away_gk_x:
            home_side = 'left'
        else:
            home_side = 'right'
        
        home_side = [home_side]

    df_list1.append([*home_tracking, *away_tracking, *home_side, ball_x, ball_y])


# Define DataFrame columns
home_tracking_columns = []
away_tracking_columns = []
for i in range(1, 24):
    home_tracking_columns.extend([f"h{i}_x", f"h{i}_y"])
    away_tracking_columns.extend([f"a{i}_x", f"a{i}_y"])
columns1 = home_tracking_columns + away_tracking_columns + ["home_side", "ball_x", "ball_y"]
df_list1 = pd.DataFrame(df_list1, columns=columns1)

df = pd.concat([df_list,df_list1],axis=1)
#Sort the DataFrame by 'period' then 'seconds'
df = df.sort_values(by=["period", "seconds"]).reset_index(drop=True)


def distance_score(pass_event, player_period, ball_velocity_period):

    event_second = pass_event[0]    
    event_player_x = pass_event[1]
    event_player_y = pass_event[2]
    event_player_side = pass_event[3]

    ball_data_dict = {
        "second": ball_velocity_period[0], 
        "x": ball_velocity_period[1], 
        "y": ball_velocity_period[2]   
    }

    tracking_event_frame = [] # 存了该second下 pass和所有tracking球员的距离与时间
    ball_distances_frame = [] #该second下所有球员和ball的距离

    tracking_second = player_period[0]    
    tracking_player_x = player_period[1]        
    tracking_player_y = player_period[2]                                                                                    
    tracking_player_id = player_period[3]

    tracking_event_distance = ((event_player_x - tracking_player_x) ** 2 + (event_player_y - tracking_player_y) ** 2) ** 0.5
    tracking_event_frame.append((tracking_second, tracking_event_distance, tracking_player_id))

    ball_distance = ((ball_data_dict["x"] - event_player_x) ** 2 + (ball_data_dict["y"] - event_player_y) ** 2) ** 0.5
    ball_distances_frame.append((tracking_second, ball_distance, tracking_player_id))


    # 计算每个球员的综合得分
    scores = []
    weight_event = 0.5
    weight_ball = 0.5

    for i in range(len(tracking_event_frame)):
        # 获取对应的时间和ID
        tracking_time_event, event_distance, player_id_event = tracking_event_frame[i]
        tracking_time_ball, ball_distance, player_id_ball = ball_distances_frame[i]

        if tracking_time_event != tracking_time_ball or player_id_event != player_id_ball:
            continue

        # 计算综合得分
        combined_score = weight_event * (1 / (event_distance + 1e-6)) + weight_ball * (1 / (ball_distance + 1e-6))
        scores.append((tracking_time_event, player_id_event, combined_score))

        # 找出最高分
        if scores:
            best_score = max(scores, key=lambda x: x[2])
            return best_score

